#### WISDOMAIN 데이터 기반 특허 분석 실습

##### 1. 데이터 준비

In [70]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Import statements required for Plotly 
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

# Import and suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [71]:
# WISDOMAIN 원본 데이터 확인
rawdata = pd.read_csv('../data/rawdata_q5.csv')
rawdata.head(1)

,번호,명칭,요약,출원번호,출원일,출원인,출원인국가,출원인대표명,현재소유자,발명자,...,자국인용특허,외국인용특허,자국피인용특허,자국인용횟수,자국피인용횟수,특허평가등급,특허평가점수,발명자수,통상실시권자,전용실시권자
0,KR20170166288A,가상 화폐 담보 대출 금융 기술 서비스 방법 및 그 장치{METHOD FOR VER...,가상 화폐 담보 대출 금융 기술 서비스 방법에서는 금융 기술 서비스 장치가 각 거래...,KR20170166288A,2017.12.05,김해동,KR,김해동,김계옥,김해동,...,"KR20110010207A,KR20170040727A,KR20170130049A","크립토코인 게시물,크립토코인 관련 20171112","KR20210083554A,KR20220049753A",3,2,A0,7.0,1,NaN,NaN


In [72]:
# 컬럼명 확인
rawdata.columns

Index(['번호', '명칭', '요약', '출원번호', '출원일', '출원인', '출원인국가', '출원인대표명', '현재소유자',
       '발명자', '발명자국가', '독립 청구항수', '전체 청구항수', '대표 청구항', '전체 청구항', '명칭(원문)',
       '요약(원문)', '국제특허분류', '공통특허분류', '자국인용특허', '외국인용특허', '자국피인용특허', '자국인용횟수',
       '자국피인용횟수', '특허평가등급', '특허평가점수', '발명자수', '통상실시권자', '전용실시권자'],
      dtype='object')

In [73]:
# 필요한 컬럼들 정의
target_cols = ['번호','명칭','요약','출원일','출원인','독립 청구항수','전체 청구항수',
               '국제특허분류','자국인용특허', '외국인용특허', '자국피인용특허', '자국인용횟수', 
               '자국피인용횟수','특허평가등급', '발명자수','발명자국가','출원인대표명']
dataset = rawdata[target_cols]
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536 entries, 0 to 535
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   번호       536 non-null    object
 1   명칭       536 non-null    object
 2   요약       536 non-null    object
 3   출원일      536 non-null    object
 4   출원인      536 non-null    object
 5   독립 청구항수  536 non-null    int64 
 6   전체 청구항수  536 non-null    int64 
 7   국제특허분류   536 non-null    object
 8   자국인용특허   451 non-null    object
 9   외국인용특허   133 non-null    object
 10  자국피인용특허  266 non-null    object
 11  자국인용횟수   536 non-null    int64 
 12  자국피인용횟수  536 non-null    int64 
 13  특허평가등급   264 non-null    object
 14  발명자수     536 non-null    int64 
 15  발명자국가    536 non-null    object
 16  출원인대표명   532 non-null    object
dtypes: int64(5), object(12)
memory usage: 71.3+ KB
None


In [74]:
# 연도가 없는 데이터 삭제
dataset.dropna(subset = ["출원일"], inplace=True)

In [75]:
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536 entries, 0 to 535
Data columns (total 17 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   번호       536 non-null    object
 1   명칭       536 non-null    object
 2   요약       536 non-null    object
 3   출원일      536 non-null    object
 4   출원인      536 non-null    object
 5   독립 청구항수  536 non-null    int64 
 6   전체 청구항수  536 non-null    int64 
 7   국제특허분류   536 non-null    object
 8   자국인용특허   451 non-null    object
 9   외국인용특허   133 non-null    object
 10  자국피인용특허  266 non-null    object
 11  자국인용횟수   536 non-null    int64 
 12  자국피인용횟수  536 non-null    int64 
 13  특허평가등급   264 non-null    object
 14  발명자수     536 non-null    int64 
 15  발명자국가    536 non-null    object
 16  출원인대표명   532 non-null    object
dtypes: int64(5), object(12)
memory usage: 75.4+ KB
None


In [76]:
def change_ipc(row):
    ipcs = [ipc.split('/')[0].split('-') for ipc in [item.strip() for item in row]]
    ipcs_revised = list(set([f"{each_ipc[0]}{each_ipc[1][1:]}" for each_ipc in ipcs]))
    res = [f"{ipc_revised[:-2]}{ipc_revised[-1]}" if ipc_revised[-2]=='0' else ipc_revised for ipc_revised in ipcs_revised ]
    return res

def count_len(row):
    if type(row) == list:
        return len(row)
    else:
        return 0
    
def remove_duplicated_list(row):
    after_row = list(set(row))
    return after_row

def extract_year(row):
    return row.split('.')[0]

In [77]:
# 새로운 컬럼들 생성
# 1. 국가
dataset.loc[dataset['번호'].str.contains('KR'),'국가'] = 'KR'
dataset.loc[dataset['번호'].str.contains('US'),'국가'] = 'US'
# 2. 연도
dataset['연도'] = dataset['출원일'].apply(extract_year)

dataset.head(1)

,번호,명칭,요약,출원일,출원인,독립 청구항수,전체 청구항수,국제특허분류,자국인용특허,외국인용특허,자국피인용특허,자국인용횟수,자국피인용횟수,특허평가등급,발명자수,발명자국가,출원인대표명,국가,연도
0,KR20170166288A,가상 화폐 담보 대출 금융 기술 서비스 방법 및 그 장치{METHOD FOR VER...,가상 화폐 담보 대출 금융 기술 서비스 방법에서는 금융 기술 서비스 장치가 각 거래...,2017.12.05,김해동,8,8,"G06Q-040/02, G06Q-020/36, G06Q-020/06","KR20110010207A,KR20170040727A,KR20170130049A","크립토코인 게시물,크립토코인 관련 20171112","KR20210083554A,KR20220049753A",3,2,A0,1,KR,김해동,KR,2017


In [78]:
dataset['국제특허분류'] = dataset['국제특허분류'].str.split(',')
dataset['외국인용횟수'] = dataset['외국인용특허'].str.split(',')
dataset['발명자국가'] = dataset['발명자국가'].str.split('|')

In [79]:
dataset['국제특허분류'] = dataset['국제특허분류'].apply(change_ipc)
dataset['외국인용횟수'] = dataset['외국인용횟수'].apply(count_len)
dataset['발명자국가'] = dataset['발명자국가'].apply(remove_duplicated_list)

In [80]:
dataset.tail(1)

,번호,명칭,요약,출원일,출원인,독립 청구항수,전체 청구항수,국제특허분류,자국인용특허,외국인용특허,자국피인용특허,자국인용횟수,자국피인용횟수,특허평가등급,발명자수,발명자국가,출원인대표명,국가,연도,외국인용횟수
535,KR20090004011U,전후 개폐가 가능한 가로등 단자함{That reorganization does av...,본 고안은 가로등주에 형성되어 가로등을 제어하는 장비의 모듈이 각각 삽입되면서 자유...,2009.04.07,유빈스 주식회사,1,4,"[F21W31, E04H12, F21S8, H02G3]","KR0254299Y1,KR0430478Y1,KR0796449B1,KR20080001...",NaN,"KR1436937B1,KR1442484B1,KR1558273B1,KR1704707B...",4,11,NaN,1,[KR],NaN,KR,2009,0


In [81]:
dataset.iloc[0]['국제특허분류']

['G06Q40', 'G06Q20']

In [82]:
dataset.to_csv("q5_dataset.csv", mode='w')